In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import os
import sys
import array


###########################################################################################

#import point cloud

#pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/model_clean.ply")
#pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/test_torso_02.ply")
#pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/1_2_weiss.ply")
pcd_01 = o3d.io.read_point_cloud("C:/Users/User/Open3D_Folder/Data/5_1.ply")

#Reduce point cloud by downsampling
print("Downsample the point cloud with a voxel of 10")
downpcd = pcd_01.voxel_down_sample(voxel_size=15)
#o3d.visualization.draw_geometries([downpcd],
#                                  zoom=0.3412,
#                                  front=[0.4257, -0.2125, -0.8795],
#                                  lookat=[2.6172, 2.0475, 1.532],
#                                  up=[-0.0694, -0.9768, 0.2024])
pcd_01 = downpcd
nr = len(pcd_01.points)
print(nr)

pcd_01_reflect = copy.deepcopy(pcd_01)

#pcd_02.paint_uniform_color([2, 0.706, 0])
#o3d.visualization.draw_geometries([pcd_01, pcd_02])

#mesh = o3d.io.read_triangle_mesh("C:/Users/User/Open3D_Folder/Data/test_torso_02.ply")
#print("Try to render a mesh with normals (exist: " +
#          str(mesh.has_vertex_normals()) + ") and colors (exist: " +
#          str(mesh.has_vertex_colors()) + ")")
#o3d.visualization.draw_geometries([mesh])
#mesh.compute_vertex_normals()
#print(np.asarray(mesh.triangle_normals))
#o3d.visualization.draw_geometries([mesh])

#vertex_normal = mesh.triangle_normals[100]
#print("Print any vertex normal")
#print(vertex_normal)

#nr = len(mesh.triangle_normals)
#print(nr)

#nr1 = len(mesh.triangles)
#print(nr1)


###########################################################################################



############################################################################################
###########################################################################################
#Definition of functions
###########################################################################################
###########################################################################################


###########################################################################################
#calculate ROI of torso back side and resample asymmetry values into matrix (X * X)
###########################################################################################

def ROI_Asymmetry_Resampling(pcd_01, pcd_01_reflect, pcd_centre_01):

    pcd_points_01 = np.asarray(pcd_01.points)
    nr = len(pcd_01.points)
    Zahl = 0
    radius = 15
    dist_raster_point = 0
    matrix_points = copy.deepcopy(pcd_points_01)
    point = copy.deepcopy(pcd_centre_01)
    temp_centre = copy.deepcopy(pcd_centre_01)

    nr_class0=0; nr_class1=0; nr_class2=0; nr_class3=0; nr_class4=0; nr_class5=0; nr_class6=0; nr_class7=0; nr_class8=0; nr_class9=0; nr_class10=0
    nr_class11=0; nr_class12=0; nr_class13=0; nr_class14=0; nr_class15=0; nr_class16=0; nr_class17=0; nr_class18=0; nr_class19=0; nr_class20=0
    nr_class21=0; nr_class22=0; nr_class23=0; nr_class24=0

    dist_class0=0; dist_class1=0 ; dist_class2=0; dist_class3=0; dist_class4=0; dist_class5=0; dist_class6=0; dist_class7=0; dist_class8=0; dist_class9=0
    dist_class10=0; dist_class11=0; dist_class12=0; dist_class13=0; dist_class14=0; dist_class15=0; dist_class16=0; dist_class17=0; dist_class18=0
    dist_class19=0; dist_class20=0; dist_class21=0; dist_class22=0; dist_class23=0; dist_class24=0

    mean_raster_point_dists = np.zeros(25, dtype='float32')

    #Calculate distance between original and reflection
    dists = pcd_01.compute_point_cloud_distance(pcd_01_reflect)
    dists = np.asarray(dists)

    #Determine torso bounding box and ROI
    boundbox = pcd_01.get_axis_aligned_bounding_box()
    box_dimensions = boundbox.get_extent()

    ROIX = box_dimensions[0]/3
    ROIY = box_dimensions[1]/3

    #Create raster points of ROI matrix 10*10
    #---------------------
    for j in range(0, 5):
        for i in range(0, 5):
            point[0] = temp_centre[0] - ROIX + ((2*ROIX)/5)*(i)
            point[1] = temp_centre[1] + ROIY - ((2*ROIY)/5)*(j)
            point[2] = 0
            matrix_points[Zahl] = point
            Zahl = Zahl + 1

    print("Anzahl Raster Points", Zahl)
    print("Torso Schwerpunkt",temp_centre[0],temp_centre[1],temp_centre[2])
    print("ROI X und Y",ROIX, ROIY)

    #Determine Torso points within radius (10mm) towards each raster point
    #and calculate mean and max raster asymmetry value
    #---------------------
    for i in range(0,nr):
      for j in range(0, Zahl):

        dist_raster_point = math.sqrt( (pcd_points_01[i,0] - matrix_points[j,0])**2 + (pcd_points_01[i,1] - matrix_points[j,1])**2 )

        if ( (dist_raster_point < radius) & (j == 0) ):
            nr_class0 = nr_class0 + 1
            dist_class0 = dist_class0 + dists[i]

        if ( (dist_raster_point < radius) & (j == 1) ):
            nr_class1 = nr_class1 + 1
            dist_class1 = dist_class1 + dists[i]

        if ( (dist_raster_point < radius) & (j == 2) ):
            nr_class2 = nr_class2 + 1
            dist_class2 = dist_class2 + dists[i]

        if ( (dist_raster_point < radius) & (j == 3) ):
            nr_class3 = nr_class3 + 1
            dist_class3 = dist_class3 + dists[i]

        if ( (dist_raster_point < radius) & (j == 4) ):
            nr_class4 = nr_class4 + 1
            dist_class4 = dist_class4 + dists[i]


        if ( (dist_raster_point < radius) & (j == 5) ):
            nr_class5 = nr_class5 + 1
            dist_class5 = dist_class5 + dists[i]

        if ( (dist_raster_point < radius) & (j == 6) ):
            nr_class6 = nr_class6 + 1
            dist_class6 = dist_class6 + dists[i]

        if ( (dist_raster_point < radius) & (j == 7) ):
            nr_class7 = nr_class7 + 1
            dist_class7 = dist_class7 + dists[i]

        if ( (dist_raster_point < radius) & (j == 8) ):
            nr_class8 = nr_class8 + 1
            dist_class8 = dist_class8 + dists[i]

        if ( (dist_raster_point < radius) & (j == 9) ):
            nr_class9 = nr_class9 + 1
            dist_class9 = dist_class9 + dists[i]


        if ( (dist_raster_point < radius) & (j == 10) ):
            nr_class10 = nr_class10 + 1
            dist_class10 = dist_class10 + dists[i]

        if ( (dist_raster_point < radius) & (j == 11) ):
            nr_class11 = nr_class11 + 1
            dist_class11 = dist_class11 + dists[i]

        if ( (dist_raster_point < radius) & (j == 12) ):
            nr_class12 = nr_class12 + 1
            dist_class12 = dist_class12 + dists[i]

        if ( (dist_raster_point < radius) & (j == 13) ):
            nr_class13 = nr_class13 + 1
            dist_class13 = dist_class13 + dists[i]

        if ( (dist_raster_point < radius) & (j == 14) ):
            nr_class14 = nr_class14 + 1
            dist_class14 = dist_class14 + dists[i]


        if ( (dist_raster_point < radius) & (j == 15) ):
            nr_class15 = nr_class15 + 1
            dist_class15 = dist_class15 + dists[i]

        if ( (dist_raster_point < radius) & (j == 16) ):
            nr_class16 = nr_class16 + 1
            dist_class16 = dist_class16 + dists[i]

        if ( (dist_raster_point < radius) & (j == 17) ):
            nr_class17 = nr_class17 + 1
            dist_class17 = dist_class17 + dists[i]

        if ( (dist_raster_point < radius) & (j == 18) ):
            nr_class18 = nr_class18 + 1
            dist_class18 = dist_class18 + dists[i]

        if ( (dist_raster_point < radius) & (j == 19) ):
            nr_class19 = nr_class19 + 1
            dist_class19 = dist_class19 + dists[i]


            
        if ( (dist_raster_point < radius) & (j == 20) ):
            nr_class20 = nr_class20 + 1
            dist_class20 = dist_class20 + dists[i]

        if ( (dist_raster_point < radius) & (j == 21) ):
            nr_class21 = nr_class21 + 1
            dist_class21 = dist_class21 + dists[i]

        if ( (dist_raster_point < radius) & (j == 22) ):
            nr_class22 = nr_class22 + 1
            dist_class22 = dist_class22 + dists[i]

        if ( (dist_raster_point < radius) & (j == 23) ):
            nr_class23 = nr_class23 + 1
            dist_class23 = dist_class23 + dists[i]

        if ( (dist_raster_point < radius) & (j == 24) ):
            nr_class24 = nr_class24 + 1
            dist_class24 = dist_class24 + dists[i]

    #end of for Schleife

    mean_raster_point_dists[0] = dist_class0/nr_class0
    mean_raster_point_dists[1] = dist_class1/nr_class1
    mean_raster_point_dists[2] = dist_class2/nr_class2
    mean_raster_point_dists[3] = dist_class3/nr_class3
    mean_raster_point_dists[4] = dist_class4/nr_class4

    mean_raster_point_dists[5] = dist_class5/nr_class5
    mean_raster_point_dists[6] = dist_class6/nr_class6
    mean_raster_point_dists[7] = dist_class7/nr_class7
    mean_raster_point_dists[8] = dist_class8/nr_class8
    mean_raster_point_dists[9] = dist_class9/nr_class9

    mean_raster_point_dists[10] = dist_class10/nr_class10
    mean_raster_point_dists[11] = dist_class11/nr_class11
    mean_raster_point_dists[12] = dist_class12/nr_class12
    mean_raster_point_dists[13] = dist_class13/nr_class13
    mean_raster_point_dists[14] = dist_class14/nr_class14

    mean_raster_point_dists[15] = dist_class15/nr_class15
    mean_raster_point_dists[16] = dist_class16/nr_class16
    mean_raster_point_dists[17] = dist_class17/nr_class17
    mean_raster_point_dists[18] = dist_class18/nr_class18
    mean_raster_point_dists[19] = dist_class19/nr_class19 

    mean_raster_point_dists[20] = dist_class20/nr_class20
    mean_raster_point_dists[21] = dist_class21/nr_class21
    mean_raster_point_dists[22] = dist_class22/nr_class22
    mean_raster_point_dists[23] = dist_class23/nr_class23
    mean_raster_point_dists[24] = dist_class24/nr_class24

    print("anzahl punkte pro raster-point")
    print(nr_class0, nr_class5, nr_class10, nr_class15, nr_class20, nr_class24)

    return mean_raster_point_dists

###########################################################################################


###########################################################################################
#calculate Cobb angle based on regression equation and 3D parameters from asymmetry analysis
###########################################################################################

def calculate_regression(param11, param22, param33, param44):

    param1 = param11
    param2 = param22
    param3 = param33
    param4 = param44

    #Calculate regression equation
    Cobb_angle = 0.5 * param1 + 0.5 * param2 + 0.5 * param3 + 0.5 * param4

    return Cobb_angle

###########################################################################################


###########################################################################################
#calculate distance histogram and aspect ratio (height / max. dist.)
###########################################################################################

def calculate_dist_histo(source_temp, target_temp):

    #define float array with ten positions for histogram classes
    class_nr_vec = np.zeros(10, dtype='float32')
    nr_class1 = 0
    nr_class2 = 0
    nr_class3 = 0
    nr_class4 = 0
    nr_class5 = 0
    nr_class6 = 0
    nr_class7 = 0
    nr_class8 = 0
    nr_class9 = 0
    nr_class10 = 0
   
    #calculate distance between the 2 torso's
    dists = source_temp.compute_point_cloud_distance(target_temp)
    dists = np.asarray(dists)
    nr = len(dists)

    #calculating maximum asymmetry distance
    max_dist = max(dists)

    #Defining 10 Histogram classes
    for i in range(0, nr):

        if ( (dists[i] > 0) & (dists[i] < 10) ):
            nr_class1 = nr_class1 + 1

        if ( (dists[i] > 10) & (dists[i] < 20) ):
            nr_class2 = nr_class2 + 1

        if ( (dists[i] > 20) & (dists[i] < 30) ):
            nr_class3 = nr_class3 + 1
        
        if ( (dists[i] > 30) & (dists[i] < 40) ):
            nr_class4 = nr_class4 + 1

        if ( (dists[i] > 40) & (dists[i] < 50) ):
            nr_class5 = nr_class5 + 1

        if ( (dists[i] > 50) & (dists[i] < 60) ):
            nr_class6 = nr_class6 + 1

        if ( (dists[i] > 60) & (dists[i] < 70) ):
            nr_class7 = nr_class7 + 1

        if ( (dists[i] > 70) & (dists[i] < 80) ):
            nr_class8 = nr_class8 + 1
        
        if ( (dists[i] > 80) & (dists[i] < 90) ):
            nr_class9 = nr_class9 + 1

        if ( (dists[i] > 90) & (dists[i] < 100) ):
            nr_class10 = nr_class10 + 1

    #Normalize all class numbers to total amount of points in pointcloud
    if (nr_class1 > 0):
        nr_class1 = nr_class1/nr
    if (nr_class2 > 0):
        nr_class2 = nr_class2/nr
    if (nr_class3 > 0):
        nr_class3 = nr_class3/nr
    if (nr_class4 > 0):
        nr_class4 = nr_class4/nr
    if (nr_class5 > 0):
        nr_class5 = nr_class5/nr
    if (nr_class6 > 0):
        nr_class6 = nr_class6/nr
    if (nr_class7 > 0):
        nr_class7 = nr_class7/nr
    if (nr_class8 > 0):
        nr_class8 = nr_class8/nr
    if (nr_class9 > 0):
        nr_class9 = nr_class9/nr
    if (nr_class10 > 0):
        nr_class10 = nr_class10/nr

    class_nr_vec[0] = nr_class1    
    class_nr_vec[1] = nr_class2
    class_nr_vec[2] = nr_class3
    class_nr_vec[3] = nr_class4
    class_nr_vec[4] = nr_class5
    class_nr_vec[5] = nr_class6    
    class_nr_vec[6] = nr_class7
    class_nr_vec[7] = nr_class8
    class_nr_vec[8] = nr_class9
    class_nr_vec[9] = nr_class10

    max_nr_class = max(class_nr_vec)*100

    #max_nr_class ist ein Wert zwischen 0-100% und max_dist die maximale Distanz in [mm]
    aspect_ratio = max_nr_class / max_dist

    print("Print the maximum distance and aspect ratio of distance histogram")
    print(max_dist,'[mm] ', aspect_ratio,'[ ]')
    print(max_nr_class)

    return aspect_ratio

###########################################################################################


###########################################################################################
#calculate distance between original and reflected torso
###########################################################################################

def calculate_asymmetry_dist(source_temp, target_temp, sign_dist_metric):
    #o3d.visualization.draw_geometries([source_temp, target_temp])

    #calculate distance between the 2 torso's
    dists = source_temp.compute_point_cloud_distance(target_temp)
    dists = np.asarray(dists)
    dists_norm = copy.deepcopy(dists) 

    #Paint point color according to distance---------------------
    lendists = len(dists)
    maxdist = max(dists)
    np_colors = np.random.rand(lendists, 3)  #create 3d float array for colors

    #assign colors normalized to 1 to RGB color array
    for i in range(0, lendists):
        #if sign_dist_metric[i] > 0:
            dists_norm[i] = dists[i]/maxdist
            np_colors[i,0] = dists_norm[i]
            np_colors[i,1] = 1-dists_norm[i]
            np_colors[i,2] = 1-dists_norm[i]
        #elif sign_dist_metric[i] < 0:
        #    np_colors[i,0] = 0
        #    np_colors[i,1] = 0
        #    np_colors[i,2] = 0

    #create temporary PointCloud pcd for visualization and assign points as well as colors 
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(source_temp.points)
    pcd.colors = o3d.utility.Vector3dVector(np_colors)
    #o3d.visualization.draw_geometries([pcd])

    #------------------------------------------------------------
    #Print Original and PointCloud with removed points (>9mm)
    #------------------------------------------------------------

    ind = np.where(sign_dist_metric > 10)[0]
    pcd_threshold = source_temp.select_by_index(ind)
    pcd_threshold.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([pcd_threshold, source_temp])

    print("Print the distances of the first 10 points")
    print(np.asarray(dists)[:10])

    #---------------------------------------------
    #Save all asymmetry distances in text file
    #---------------------------------------------
    print('Save distances in float array')
    #---------------------------------------------
    lendists = len(dists)
    f = open("distances.dat", "w+")
    for i in range(0, lendists):
        f.write(str(sign_dist_metric[i]))
        f.write("\n")  
    f.close()
    #----------------------------------------------

#---------------------------------------------------------------------------


###########################################################################################
#calculate statistics (max. asymmetry distance, max. surface rotation, patch area > 9mm)
###########################################################################################

def calculate_statistics(pcd_01, sign_dist_metric, pcd_centre_01):

    #Diese Funktion bestimmt klinische Asymmetrieparameter:
    #max. Asymmetriedistanz, MaxSurfaceNormal, PatchArea, LatDev, AspectRatio

    parameter_results = np.zeros(3, dtype='float32')

    pcd_01.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=30, max_nn=30))

    boundbox = pcd_01.get_axis_aligned_bounding_box()
    box_dimensions = boundbox.get_extent()
    print("Bounding Box Dimensions X, Y, Z: " ,box_dimensions)

    ROIX = box_dimensions[0]/3
    ROIY = box_dimensions[1]/3

    #o3d.visualization.draw_geometries([pcd_01],
    #                              zoom=0.3412,
    #                              front=[0.4257, -0.2125, -0.8795],
    #                              lookat=[2.6172, 2.0475, 1.532],
    #                              up=[-0.0694, -0.9768, 0.2024],
    #                              point_show_normal=True)

    #print("any normal vector")
    #print(print(pcd_01.normals[10]))

    #Variablen initialisieren
    pcd_points_01 = np.asarray(pcd_01.points)
    pcd_patch_points = copy.deepcopy(pcd_points_01)
    sign_dist_metric_patch = copy.deepcopy(sign_dist_metric)
    pcd_normals_ = pcd_01.normals
    pcd_patch_normals = copy.deepcopy(pcd_normals_)
    #vec_normal = pcd_points_01[1,:]
    nr = len(pcd_points_01)

    
    distmax = -100000
    surf_angle_max = 0
    thresh = 9
    Zahl = 0

    ###################################################################################
    #Bestimmt alle Punkte mit Asymmetrie > 9mm also die Asymmetrie Patches
    #aber nur wenn Punkte in ROI am Ruecken liegen: Rechteck: X-Y: 200*500mm
    #---------------------
    for i in range(nr):
        #alle Punkte mit negativer Z Koordinate [i,2] und sign_dist > 9mm
        if (pcd_points_01[i,2] < 0) & (pcd_points_01[i,0] < ROIX) & (pcd_points_01[i,0] > -ROIX) & (pcd_points_01[i,1] < ROIY) & (pcd_points_01[i,1] > -ROIY) & (sign_dist_metric[i] > thresh):
            Zahl = Zahl + 1
            #pcd_patch_normals[i,1] = 0
            #legt neue arrays an fuer patch
            pcd_patch_points[Zahl]=pcd_points_01[i]
            sign_dist_metric_patch[Zahl] = sign_dist_metric[i]
            pcd_patch_normals[Zahl] = pcd_normals_[i]

    #Local patch point number devided by total torso point number
    patch_anteil = (Zahl/nr)*100
    nr_patch = Zahl

    #Bestimmt die maximale Oberflaechen Rotation im Patch und die maximale Asymmetrie Distanz
    for i in range(nr_patch):
        if sign_dist_metric_patch[i] > distmax: 
            distmax = sign_dist_metric_patch[i]

        ####################################################################################

        vec_normal = pcd_patch_normals[i]
        vec_normal = np.asarray(vec_normal)
        #print("normal vectors")
        #print(vec_normal)

        #Calculate surface rotation relative to sagittal plane
        #Normalen Vektor in Sagittal Achse
        ortho_vec = copy.deepcopy(vec_normal)
        ortho_vec[0] = 0
        ortho_vec[1] = 0
        ortho_vec[2] = -1

        #print("ortho_vec")
        #print(ortho_vec)

        #calculate vector norm
        vec_normal_norm  = math.sqrt( vec_normal[0]**2 + vec_normal[1]**2 + vec_normal[2]**2 )
        ortho_vec_norm = math.sqrt( ortho_vec[0]**2 + ortho_vec[1]**2 + ortho_vec[2]**2 )

        #print(vec_normal_norm)
        #print(ortho_vec_norm)

        #dot product of vectors
        
        dot_product_vectors = vec_normal[0]*ortho_vec[0] + vec_normal[1]*ortho_vec[1] + vec_normal[2]*ortho_vec[2] 
        #print(dot_product_vectors)

        #Temp = 3.14 * ( (dot_product_vectors) / (vec_normal_norm * ortho_vec_norm) / 180 )
        Temp = ( (dot_product_vectors) / (vec_normal_norm * ortho_vec_norm) )
        #Umrechnung Bogenmaß in Grad
        surf_angle = (math.acos( Temp )) * (180/3.14)

        if surf_angle > surf_angle_max: 
            surf_angle_max = copy.deepcopy(surf_angle)

    print("This is the maximum surface rotation within the ROI and the patch (distance > 9mm) !")
    print(surf_angle_max)

    print('maxDist / maxSurfAngle / PatchArea')
    print(distmax,'[mm]' ,surf_angle_max, '[deg] ', patch_anteil,'[%]')

    parameter_results[0] = distmax
    parameter_results[1] = surf_angle_max
    parameter_results[2] = patch_anteil

    return parameter_results


###########################################################################################
#ICP matching for original and reflection
###########################################################################################

###########################################################################################

#type(source)

#------------------------------------------------------------------------
def draw_registration_result_init(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


#------------------------------------------------------------------------
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

    calculate_asymmetry_dist(source_temp, target_temp, sign_dist_metric)


#--------------------------------------------------------------------------
def point_to_point_icp(source, target, threshold, trans_init):
    print("Apply point-to-point ICP")
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())

    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation, "\n")
    draw_registration_result_init(source, target, reg_p2p.transformation)

#-------------------------------------------------------------------------
def point_to_plane_icp(source, target, threshold, trans_init):
    print("Apply point-to-plane ICP")
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())

    print(reg_p2l)
    print("Transformation is:")
    print(reg_p2l.transformation, "\n")
    draw_registration_result(source, target, reg_p2l.transformation)

#---------------------------------------------------------------------------
###########################################################################################


###########################################################################################
###########################################################################################
#End of function definition
###########################################################################################
###########################################################################################





#Estimate vertex normals

#it is important to chose the right radius (not to small >5) to gain realistic surface normals
pcd_01.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
        radius=10, max_nn=30))

#o3d.visualization.draw_geometries([pcd_01],
#                                  zoom=0.3412,
#                                  front=[0.4257, -0.2125, -0.8795],
#                                  lookat=[2.6172, 2.0475, 1.532],
#                                  up=[-0.0694, -0.9768, 0.2024],
#                                  point_show_normal=True)

print("Print a normal vector of the 0th point")
print(pcd_01.normals[0])

print("Print the normal vectors of the first 10 points")
print(np.asarray(pcd_01.normals)[:10, :])

nr1 = len(pcd_01.normals)
print(nr1)

##############################################################################################
##############################################################################################
##############################################################################################

###########################################################################################
#oriented bounding box

#obj1 = pcd_01.get_axis_aligned_bounding_box()
#obj1.color = (1, 0, 0)
#obj2 = pcd_01.get_oriented_bounding_box()
#obj2.color = (0, 0, 1)
#o3d.visualization.draw_geometries([pcd_01, obj1, obj2])

##########################################################################################
#calculate centre of point cloud
##########################################################################################

pcd_points_01 = np.asarray(pcd_01.points)
pcd_centre_01 = pcd_points_01[1,:] + pcd_points_01[2,:]
pcd_centre_01[0] = 0
pcd_centre_01[1] = 0
pcd_centre_01[2] = 0

#calculate centre of torso point cloud####################################################

nr = len(pcd_points_01)
print(nr)

#calculate centre
#---------------------
for i in range(nr):
  pcd_centre_01 = pcd_centre_01 + pcd_points_01[i,:]
#---------------------
pcd_centre_01 = pcd_centre_01/nr

#centre point cloud # origin (0,0,0) is placed in torso centre
#---------------------
for i in range(nr):
  pcd_points_01[i,:] = pcd_points_01[i,:] - pcd_centre_01

print(np.asarray(pcd_points_01)[0,:])

##########################################################################################
#make copy of point cloud and perform reflection about sagittal plane
##########################################################################################

#make copy of torso object
#pcd_points_01_reflect = pcd_points_01
pcd_points_01_reflect = pcd_points_01.copy()
#print(np.asarray(pcd_centre_01)[:])
    #source_temp = copy.deepcopy(source)
    #target_temp = copy.deepcopy(target)

#do reflection about sagittal plane
for i in range(nr):
  temp = pcd_points_01[i,0]
  pcd_points_01_reflect[i,0] = -temp

print(np.asarray(pcd_points_01)[0,:])
print(np.asarray(pcd_points_01_reflect)[0,:])


##########################################################################

##########################################################################
##########################################################################
#Assign centred and reflected points again to point cloud objects
##########################################################################

pcd_01.points = o3d.utility.Vector3dVector(pcd_points_01)
pcd_01_reflect.points = o3d.utility.Vector3dVector(pcd_points_01_reflect)

##########################################################################
##########################################################################

##########################################################################

###########################################################################################
#------------------------------------------------------------------------------------------
#Detect which points of the reflection are positioned inside the original volume
#points with shorter distance to the centre are inside
#------------------------------------------------------------------------------------------
###########################################################################################

#Variablennamen ueberpruefen

sign_dist = pcd_points_01[:,0].copy()
sign_dist_metric = pcd_points_01[:,0].copy()

for i in range(nr):
    distmin = 100000

    #Distanzfunktion von Open3D nutzen anstatt Schleife
    for k in range(math.floor(nr/1)): #werte jeden 30. Punkt aus
        
        j = k*1
#        #calculate manually the distance between original and reflection
        dists_man = math.sqrt( (pcd_points_01[i,0] - pcd_points_01_reflect[j,0])**2 + (pcd_points_01[i,1] - pcd_points_01_reflect[j,1])**2 + (pcd_points_01[i,2] - pcd_points_01_reflect[j,2])**2 )

        if dists_man < distmin: 
            distmin = dists_man
            near_point = pcd_points_01_reflect[j,:]
            sign_dist_metric[i] = dists_man

    #Calculate the distance between torso points and centre of mass for original and reflection
    dist_orig = math.sqrt( (pcd_points_01[i,0] - pcd_centre_01[0])**2 + (pcd_points_01[i,1] - pcd_centre_01[1])**2 + (pcd_points_01[i,2] - pcd_centre_01[2])**2 )
    dist_refl = math.sqrt( (near_point[0] - pcd_centre_01[0])**2 + (near_point[1] - pcd_centre_01[1])**2 + (near_point[2] - pcd_centre_01[2])**2 )

    if dist_orig > dist_refl:
        sign_dist[i] = 1
    else: 
        sign_dist[i] = 0                            #Vorzeichen der Asymmetrie Distanz +/- zwischen Original und Reflection
        sign_dist_metric[i] = - sign_dist_metric[i] #Metrischer Abstand [mm] zwischen Original und Reflection

print("Print the signed distances of the first 10 points")
print(np.asarray(sign_dist)[:])


###########################################################################################
#Start of matching
###########################################################################################

#if __name__ == "__main__":
    #pcd_data = o3d.data.DemoICPPointClouds()
    #source = o3d.io.read_point_cloud(pcd_data.paths[0])
    #source_final = copy.deepcopy(source)
    #target = o3d.io.read_point_cloud(pcd_data.paths[1])
    # Input point-clouds
    #pcd_points_01_reflect = pcd_points_01
#Wichtig ist, dass hier 3D point cloud objekte fuer das Matching uebergeben werden
#nur die Punktkoordinaten reichen nicht aus
source = copy.deepcopy(pcd_01)
target = copy.deepcopy(pcd_01_reflect)
#threshold = 0.105
threshold = 70 
    #trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
    #                         [-0.139, 0.967, -0.215, 0.7],
    #                         [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])
    
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0], [0.0, 0.0, 0.0, 1.0]])
    


    #draw_registration_result_init(source, target, trans_init)
    #print("Initial alignment")

evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)

evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)

evaluation = o3d.pipelines.registration.evaluate_registration(
        source, target, threshold, trans_init)

print(evaluation, "\n")
    #point_to_point_icp(source, target, threshold, trans_init)
point_to_plane_icp(source, target, threshold, trans_init)


################################################################################
#Start of statitistic calculation based on asymmetry map
################################################################################

parameters0 = calculate_dist_histo(source, target)

parameters1 = calculate_statistics(pcd_01, sign_dist_metric, pcd_centre_01)

#print evaluation parameters
print('This are the calculated predicted asymmetry parameters:')
print("aspect ratio [ ]", parameters0, "\n")     #aspect ratio 
print("max distance [mm]",parameters1[0], "\n")  #max distance mm
print("surf angle [deg]",parameters1[1], "\n")  #surf angle deg
print("patch area [%]", parameters1[2], "\n")  #patach area %

##################################################################################
#Save parameters for further evaluation -> training data set for statistical model
##################################################################################

print('Save statistical parameters')
#--------------------------------------------
f = open("parameters.dat", "w+")
f.write(str(parameters0))
f.write("\n")  
f.write(str(parameters1[0]))
f.write("\n")  
f.write(str(parameters1[1]))
f.write("\n")  
f.write(str(parameters1[2]))
f.close()
#--------------------------------------------

Cobb_angle = calculate_regression(parameters0, parameters1[0],parameters1[1], parameters1[2])

print("This is the predicted Cobb angle: ",Cobb_angle)     #aspect ratio 


mean_raster_point_dists = ROI_Asymmetry_Resampling(pcd_01, pcd_01_reflect, pcd_centre_01)

print("Print the mean asymmetry distances around each raster point 5*5 matrix")
print(np.asarray(mean_raster_point_dists)[:24])

##################################################################################
#Save pAsymmetry values for each raster point for evaluation: Training CNN
##################################################################################

print('Save mean asymmetry values for raster points')
#--------------------------------------------
f = open("raster_point_asymmetries.dat", "w+")
for i in range(24):
    f.write(str(mean_raster_point_dists[i]))
    f.write("\n")  
f.close()
#--------------------------------------------








Downsample the point cloud with a voxel of 10
3559
Print a normal vector of the 0th point
[-0. -0. -1.]
Print the normal vectors of the first 10 points
[[-0.         -0.         -1.        ]
 [ 0.          0.          1.        ]
 [-0.08773227 -0.97344887 -0.21142455]
 [-0.         -0.         -1.        ]
 [ 0.          0.          1.        ]
 [-0.         -0.         -1.        ]
 [ 0.          0.          1.        ]
 [-0.         -0.         -1.        ]
 [ 0.          0.          1.        ]
 [-0.         -0.         -1.        ]]
3559
3559
[-124.53348729 -151.53862753  -79.97650876]
[-124.53348729 -151.53862753  -79.97650876]
[ 124.53348729 -151.53862753  -79.97650876]
Print the signed distances of the first 10 points
[0. 1. 0. ... 0. 0. 0.]
RegistrationResult with fitness=1.000000e+00, inlier_rmse=1.391133e+01, and correspondence_set size of 3559
Access transformation to get result. 

Apply point-to-plane ICP
RegistrationResult with fitness=1.000000e+00, inlier_rmse=9.473555e+0